In [ ]:
!python train.py --epochs 1 --train-photo-path train_photo_test --dataset shinkai_test --batch 4 --init-epochs 4 --use_sn 

In [1]:
import torch.optim as optim
from utils.common import load_checkpoint
from modeling.anime_gan import Generator
lr=0.001
G = Generator('shinkai').to('cpu')

start_e = load_checkpoint(G, 'checkpoint')
optimizer_g = optim.Adam(G.parameters(), lr=0.001, betas=(0.5, 0.999))
for param_group in optimizer_g.param_groups:
    param_group['lr']

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint\\generator_shinkai.pth'

In [15]:
start_e

9